In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_cleaning import load_trade

In [46]:
Date = pd.bdate_range('2010-01-01','2010-12-31')
Market_name = np.array(['AAPL.OQ','AMGN.OQ','AXP.N','BA.N','CAT.N','CSCO.OQ','CVX.N','DOW.N','GS.N','SPY.P','UTX.N','V.N','WMT.N'])

#folder_path = 'D:/GitHub/ML_For_Finance/big data/data/data/'

folder_path = '/Users/franckatteaka/Desktop/cours/Semester III/Financial big data/high freq data/'

In [50]:
load_trade(Market_name[0],Date[2],folder_path,is_compressed = True)

,AAPL.OQ
xltime,
2010-01-05 09:30:00.009014400-05:00,214.50
2010-01-05 09:30:00.009014400-05:00,214.50
2010-01-05 09:30:00.009014400-05:00,214.51
2010-01-05 09:30:00.009014400-05:00,214.52
2010-01-05 09:30:00.009014400-05:00,214.54
...,...
2010-01-05 15:59:59.482032-05:00,214.37
2010-01-05 15:59:59.482032-05:00,214.37
2010-01-05 15:59:59.792985600-05:00,214.35


In [15]:
load_trade(Market_name[0],Date[2],path_big_data2,path_perso2)

,AAPL.OQ
xltime,
2010-01-05 09:30:00.009014400-05:00,214.50
2010-01-05 09:30:00.009014400-05:00,214.50
2010-01-05 09:30:00.009014400-05:00,214.51
2010-01-05 09:30:00.009014400-05:00,214.52
2010-01-05 09:30:00.009014400-05:00,214.54
...,...
2010-01-05 15:59:59.482032-05:00,214.37
2010-01-05 15:59:59.482032-05:00,214.37
2010-01-05 15:59:59.792985600-05:00,214.35


In [16]:
X1=load_trade(Market_name[0],Date[2],path_big_data,path_perso)
X2=load_trade(Market_name[1],Date[2],path_big_data,path_perso)

In [18]:
X = pd.concat([X1,X2])
X.iloc[:100]

,AAPL.OQ,AMGN.OQ
xltime,,
2010-01-05 09:30:00.009014400-05:00,214.50,NaN
2010-01-05 09:30:00.009014400-05:00,214.50,NaN
2010-01-05 09:30:00.009014400-05:00,214.51,NaN
2010-01-05 09:30:00.009014400-05:00,214.52,NaN
2010-01-05 09:30:00.009014400-05:00,214.54,NaN
...,...,...
2010-01-05 09:30:05.294016-05:00,214.45,NaN
2010-01-05 09:30:05.296003200-05:00,214.45,NaN
2010-01-05 09:30:05.306976-05:00,214.48,NaN


In [22]:
X1[10:50]

,AAPL.OQ
xltime,
2010-01-05 09:30:00.160041600-05:00,214.53
2010-01-05 09:30:00.234000-05:00,214.54
2010-01-05 09:30:00.234000-05:00,214.53
2010-01-05 09:30:00.293011200-05:00,214.58
2010-01-05 09:30:00.336038400-05:00,214.58
2010-01-05 09:30:00.336038400-05:00,214.58
2010-01-05 09:30:00.347961600-05:00,214.56
2010-01-05 09:30:00.399023999-05:00,214.62
2010-01-05 09:30:00.399023999-05:00,214.62


In [20]:
X2

,AMGN.OQ
xltime,
2010-01-05 09:30:00.368956800-05:00,57.40
2010-01-05 09:30:01.181980800-05:00,57.40
2010-01-05 09:30:01.820995200-05:00,57.40
2010-01-05 09:30:02.057990400-05:00,57.40
2010-01-05 09:30:02.205993600-05:00,57.49
...,...
2010-01-05 15:59:59.982979200-05:00,57.24
2010-01-05 15:59:59.982979200-05:00,57.24
2010-01-05 15:59:59.982979200-05:00,57.24


In [21]:
X

,AAPL.OQ,AMGN.OQ
xltime,,
2010-01-05 09:30:00.009014400-05:00,214.50,NaN
2010-01-05 09:30:00.009014400-05:00,214.50,NaN
2010-01-05 09:30:00.009014400-05:00,214.51,NaN
2010-01-05 09:30:00.009014400-05:00,214.52,NaN
2010-01-05 09:30:00.009014400-05:00,214.54,NaN
...,...,...
2010-01-05 15:59:59.982979200-05:00,NaN,57.24
2010-01-05 15:59:59.982979200-05:00,NaN,57.24
2010-01-05 15:59:59.982979200-05:00,NaN,57.24
